In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 32.6 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd 

import cv2
import pydicom
from PIL import Image
from IPython.display import Image as IPyImage, display

import os
import re
import glob
import random
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import train_test_split

from torchvision import transforms
import timm

import yaml

import albumentations as A

from sklearn.model_selection import KFold

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
train_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
label_coords_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')
series_desc_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

In [4]:
# Getting a list of all the study IDs and paths to their images
images_dir_path = r'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
study_id_list = os.listdir(images_dir_path)
study_id_paths = [(x, f"{images_dir_path}/{x}") for x in study_id_list]

# Initialize the metadata dictionary
meta_df = {}

# Process each study and its series
for study_id, study_folder_path in study_id_paths:
    series_ids = []
    series_descriptions = []
    
    # Get all the series IDs (folders) within the study folder
    try:
        series_folders = os.listdir(study_folder_path)
    except FileNotFoundError as e:
        print(f"Error: Folder not found for study {study_id}. Skipping this study.")
        continue  # Skip this study if the folder doesn't exist

    # Process each series in the study folder
    for series_id in series_folders:
        try:
            # Fetch the series description from the dataframe
            series_description = series_desc_df[series_desc_df['series_id'] == int(series_id)]['series_description'].iloc[0]
        except (IndexError, ValueError):
            # Handle cases where series_id is not found in the dataframe or can't be converted to int
            series_description = 'Unknown'

        # Append series ID and description to the lists
        series_ids.append(series_id)
        series_descriptions.append(series_description)
    
    # Add metadata for the current study_id
    meta_df[int(study_id)] = {
        'folder_path': study_folder_path,
        'series_ids': series_ids,
        'series_descriptions': series_descriptions
    }

In [5]:
def add_desc(df, meta_df):
    df['series_desc'] = None

    # Iterate over rows in the dataframe
    for idx, coor_row in df.iterrows():
        try:
            # Find the meta_df for the study_id
            meta_info = meta_df[int(coor_row['study_id'])]

            # Find the index of the series_id in the meta_info
            series_index = meta_info['series_ids'].index(str(coor_row['series_id']))

            # Get the corresponding series description
            series_desc = meta_info['series_descriptions'][series_index]

            # Update the series_desc column
            df.at[idx, 'series_desc'] = series_desc

        except KeyError:
            print(f"Error processing study_id: {coor_row['study_id']} - Study ID not found in meta_df")
            df.at[idx, 'series_desc'] = 'Unknown'
        except ValueError:
            print(f"Error processing study_id: {coor_row['study_id']} - Series ID not found in meta_df")
            df.at[idx, 'series_desc'] = 'Unknown'
        except Exception as e:
            print(f"Error processing study_id: {coor_row['study_id']} - {e}")
            df.at[idx, 'series_desc'] = 'Unknown'
    
    return df

# Apply the function
coords_with_desc = label_coords_df.copy()
coords_with_desc = add_desc(coords_with_desc, meta_df)
coords_with_desc.head(20)

,study_id,series_id,instance_number,condition,level,x,y,series_desc
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR
5,4003253,1054713880,4,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,Sagittal T1
6,4003253,1054713880,4,Right Neural Foraminal Narrowing,L5/S1,198.240918,285.613767,Sagittal T1
7,4003253,1054713880,5,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,Sagittal T1
8,4003253,1054713880,6,Right Neural Foraminal Narrowing,L1/L2,194.569790,127.755258,Sagittal T1
9,4003253,1054713880,6,Right Neural Foraminal Narrowing,L2/L3,191.632887,165.934990,Sagittal T1


In [6]:
sagt1_df = coords_with_desc[coords_with_desc['series_desc'] == 'Sagittal T1'].copy()
sagt2_df = coords_with_desc[coords_with_desc['series_desc'] == 'Sagittal T2/STIR'].copy()
axialt2_df = coords_with_desc[coords_with_desc['series_desc'] == 'Axial T2'].copy()

In [7]:
class SagT2_YOLO:
    def __init__(self, df, images_dir, output_dir, img_size=384):
        """
        Initialize Sagittal T2/STIR YOLO detector
        Args:
            df: DataFrame with annotations
            images_dir: Path to DICOM images
            output_dir: Path to save processed dataset
            img_size: Target image size for YOLO
        """
        self.df = df
        self.images_dir = images_dir
        self.output_dir = output_dir
        self.img_size = img_size
        
        # Create initial directory structure
        self.create_dataset_structure()
        
        # Process dataset
        self.instance_coords_df = self.process_instance_coordinates()
        self.processed_df = self.process_spine_dataset()
        
        # Create YAML and train model
        self.yaml_path = self.create_dataset_yaml()
        self.model, self.results = self.train_yolo()

    def create_dataset_structure(self):
        """Create YOLO dataset directory structure"""
        for split in ['train', 'val']:
            for subdir in ['images', 'labels']:
                path = os.path.join(self.output_dir, split, subdir)
                os.makedirs(path, exist_ok=True)

    def process_instance_coordinates(self):
        """
        Process coordinates for Sagittal T2 images with coordinate sharing across instances
        Returns DataFrame with coordinates for all instances, sharing information across the series
        """
        result_records = []

        # Group by series to process related instances
        for (study_id, series_id), series_data in self.df.groupby(['study_id', 'series_id']):
            # First, collect all coordinates for each level in the series
            series_level_coords = {}
            for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
                level_data = series_data[series_data['level'] == level]

                if not level_data.empty:
                    coords = []
                    instances_with_data = set()  # Track which instances have data for this level

                    for _, row in level_data.iterrows():
                        instances_with_data.add(row['instance_number'])
                        coords.append((row['x'], row['y']))

                    # Calculate average coordinates for this level
                    if coords:
                        avg_x = np.mean([x for x, _ in coords])
                        avg_y = np.mean([y for _, y in coords])
                        
                        series_level_coords[level] = {
                            'instances': instances_with_data,
                            'coords': (avg_x, avg_y),
                            'original_coords': coords  # Keep original coordinates for reference
                        }

            # Get all unique instance numbers in the series
            all_instances = series_data['instance_number'].unique()

            # Create records for each instance
            for instance_number in all_instances:
                instance_data = series_data[series_data['instance_number'] == instance_number]
                
                record = {
                    'study_id': study_id,
                    'series_id': series_id,
                    'instance_number': instance_number,
                    'coordinate_sources': {}  # Track where coordinates came from
                }

                # Process each spinal level
                for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
                    level_key = level.replace('/', '_').lower()
                    
                    # Check if this instance has original coordinates for this level
                    instance_level_data = instance_data[instance_data['level'] == level]
                    
                    if not instance_level_data.empty:
                        # Use original coordinates for this instance
                        x = instance_level_data.iloc[0]['x']
                        y = instance_level_data.iloc[0]['y']
                        record['coordinate_sources'][level_key] = 'original'
                    elif level in series_level_coords:
                        # Use shared coordinates from series
                        x, y = series_level_coords[level]['coords']
                        record['coordinate_sources'][level_key] = 'shared'
                    else:
                        # No coordinates available
                        x = y = None
                        record['coordinate_sources'][level_key] = 'missing'
                    
                    record[f'{level_key}_x'] = x
                    record[f'{level_key}_y'] = y

                result_records.append(record)

        # Convert to DataFrame
        result_df = pd.DataFrame(result_records)

        # Add metadata about coordinate availability
        result_df['available_levels'] = result_df.apply(
            lambda row: [
                level for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']
                if not pd.isna(row[f"{level.replace('/', '_').lower()}_x"])
            ],
            axis=1
        )
        
        result_df['total_levels'] = result_df['available_levels'].apply(len)

        # Print statistics
        print("\nDataset Statistics:")
        print(f"Total series processed: {len(result_df['series_id'].unique())}")
        print(f"Total instances processed: {len(result_df)}")
        print("\nLevel availability:")
        for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
            count = result_df[f'{level}_x'].notna().sum()
            original_count = sum(result_df['coordinate_sources'].apply(
                lambda x: x.get(level, '') == 'original'
            ))
            shared_count = sum(result_df['coordinate_sources'].apply(
                lambda x: x.get(level, '') == 'shared'
            ))
            print(f"{level}: {count} instances ({count/len(result_df)*100:.1f}%)")
            print(f"  - Original: {original_count}")
            print(f"  - Shared: {shared_count}")

        return result_df

    def create_yolo_annotation(self, row, image_width, image_height):
        """Create YOLO format annotations for Sagittal T2 images"""
        annotations = []
        box_width = 0.05  # Relative box width
        box_height = 0.05  # Relative box height
        
        level_map = {
            'l1_l2': 0,
            'l2_l3': 1,
            'l3_l4': 2,
            'l4_l5': 3,
            'l5_s1': 4
        }
        
        for level, idx in level_map.items():
            x_coord = row.get(f'{level}_x')
            y_coord = row.get(f'{level}_y')
            
            if pd.notna(x_coord) and pd.notna(y_coord):
                x_norm = x_coord / image_width
                y_norm = y_coord / image_height
                annotations.append(f"{idx} {x_norm:.6f} {y_norm:.6f} {box_width:.6f} {box_height:.6f}")
        
        return annotations

    def process_spine_dataset(self):
        """Process and save dataset in YOLO format"""
        # Split studies
        studies = self.instance_coords_df['study_id'].unique()
        train_studies, val_studies = train_test_split(studies, train_size=0.8, random_state=42)
        
        processed_counts = {'train': 0, 'val': 0}
        failed_cases = []
        
        for _, row in tqdm(self.instance_coords_df.iterrows(), desc="Processing Sagittal T1 images"):
            try:
                # Convert IDs to integers for path construction
                study_id = str(int(row['study_id']))
                series_id = str(int(row['series_id']))
                instance_number = str(int(row['instance_number']))
                
                # Construct image path
                img_path = os.path.join(self.images_dir, study_id, series_id, instance_number)
                
                # Try with and without .dcm extension
                if os.path.exists(img_path + '.dcm'):
                    img_path = img_path + '.dcm'
                elif not os.path.exists(img_path):
                    raise FileNotFoundError(f"Image not found: {img_path}")
                
                # Read and process image
                ds = pydicom.dcmread(img_path)
                image = ds.pixel_array
                h, w = image.shape
                
                # Create annotations
                annotations = self.create_yolo_annotation(row, w, h)
                if not annotations:
                    continue
                
                # Prepare image
                image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                image_resized = cv2.resize(image_normalized, (self.img_size, self.img_size))
                
                # Save files
                is_train = row['study_id'] in train_studies
                split = 'train' if is_train else 'val'
                
                img_filename = f"{study_id}_{series_id}_{instance_number}.png"
                label_filename = f"{study_id}_{series_id}_{instance_number}.txt"
                
                cv2.imwrite(os.path.join(self.output_dir, split, 'images', img_filename), 
                           image_resized)
                with open(os.path.join(self.output_dir, split, 'labels', label_filename), 'w') as f:
                    f.write('\n'.join(annotations))
                
                # Add split information to row
                row['split'] = split
                processed_counts[split] += 1
                
            except Exception as e:
                failed_cases.append((study_id, series_id, instance_number, str(e)))
        
        print(f"\nProcessing Summary:")
        print(f"Training images: {processed_counts['train']}")
        print(f"Validation images: {processed_counts['val']}")
        
        if failed_cases:
            print("\nFailed cases:")
            for case in failed_cases:
                print(f"Study {case[0]}, Series {case[1]}, Instance {case[2]}: {case[3]}")
        
        return self.instance_coords_df

    def create_dataset_yaml(self):
        """Create YOLO dataset configuration file"""
        yaml_content = {
            'path': os.path.abspath(self.output_dir),
            'train': 'train/images',
            'val': 'val/images',
            'nc': 5,  # number of classes
            'names': {
                0: 'L1/L2',
                1: 'L2/L3',
                2: 'L3/L4',
                3: 'L4/L5',
                4: 'L5/S1'
            }
        }

        yaml_path = os.path.join(self.output_dir, 'dataset.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, sort_keys=False)

        return yaml_path

    def train_yolo(self):
        """Train YOLO model for Sagittal T1 images"""
        try:
            model = YOLO('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11x.pt')
            
            config = {
                'data': self.yaml_path,
                'imgsz': self.img_size,
                'batch': 16,
                'epochs': 100,
                'patience': 15,
                'device': '0',
                'workers': 8,
                'project': 'spine_detection',
                'name': 'sagittal_t2_yolo',
                'exist_ok': True,
                'pretrained': True,
                'optimizer': 'AdamW',
                'verbose': True,
                'seed': 42,
                'deterministic': True,
                'dropout': 0.2,
                'lr0': 0.001,
                'lrf': 0.01,
                'momentum': 0.937,
                'weight_decay': 0.0005,
                'warmup_epochs': 10,
                'warmup_momentum': 0.8,
                'box': 7.5,
                'cls': 0.5,
                'dfl': 1.5,
                'close_mosaic': 10,
                'amp': True,
            }
            
            results = model.train(**config)
            return model, results
            
        except Exception as e:
            print(f"Error training model: {str(e)}")
            return None, None

    def get_training_stats(self):
        """Get statistics about the processed dataset"""
        stats = {
            'total_series': len(self.instance_coords_df['series_id'].unique()),
            'total_instances': len(self.instance_coords_df),
            'train_images': len(self.processed_df[self.processed_df['split'] == 'train']),
            'val_images': len(self.processed_df[self.processed_df['split'] == 'val']),
            'level_coverage': {}
        }
        
        for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
            count = self.instance_coords_df[f'{level}_center_x'].notna().sum()
            stats['level_coverage'][level] = {
                'count': count,
                'percentage': count/len(self.instance_coords_df)*100
            }
            
        return stats

In [ ]:
# Sagittal T2/STIR
images_dir  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
output_dir = '/kaggle/working/spine_dataset_segt2'

sag_t2_model = SagT2_YOLO(
    df=sagt2_df,
    images_dir=images_dir,
    output_dir=output_dir,
    img_size=384
)


Dataset Statistics:
Total series processed: 1973
Total instances processed: 2521

Level availability:
l1_l2: 2441 instances (96.8%)
  - Original: 1903
  - Shared: 538
l2_l3: 2478 instances (98.3%)
  - Original: 1933
  - Shared: 545
l3_l4: 2520 instances (100.0%)
  - Original: 1972
  - Shared: 548
l4_l5: 2519 instances (99.9%)
  - Original: 1971
  - Shared: 548
l5_s1: 2517 instances (99.8%)
  - Original: 1969
  - Shared: 548


Processing Sagittal T1 images: 2521it [00:49, 50.62it/s]


Processing Summary:
Training images: 2018
Validation images: 503



100%|██████████| 109M/109M [00:00<00:00, 266MB/s] 


Ultralytics 8.3.48 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=weights/yolo11x.pt, data=/kaggle/working/spine_dataset_segt2/dataset.yaml, epochs=100, time=None, patience=15, batch=16, imgsz=384, save=True, save_period=-1, cache=False, device=0, workers=8, project=spine_detection, name=sagittal_t2_yolo, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     32/100      7.35G      1.631     0.9669     0.9157         12        384: 100%|██████████| 127/127 [01:19<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:07<00:00,  2.10it/s]

                   all        503       2481      0.907      0.908      0.908      0.424



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     33/100      7.26G      1.623     0.9873     0.9044        115        384:  23%|██▎       | 29/127 [00:18<01:01,  1.59it/s]

In [ ]:
def save_trained_model(model, best_model_path, model_type, save_path='/kaggle/working/'):
    """
    Save the trained YOLO model with simple fixed naming
    
    Args:
        model: YOLO model object
        best_model_path: Path to the best model weights
        model_type: String indicating model type ('sag_t1', 'sag_t2', or 'axial_t2')
        save_path: Base path to save the model
    """
    # Define simple model names
    model_names = {
        'sag_t1': 'sagittal_t1_spine_detector.pt',
        'sag_t2': 'sagittal_t2_spine_detector.pt',
        'axial_t2': 'axial_t2_spine_detector.pt'
    }
    
    if model_type not in model_names:
        raise ValueError(f"Invalid model_type: {model_type}. Must be one of {list(model_names.keys())}")
    
    try:
        if not os.path.exists(best_model_path):
            print(f"Best model weights not found at {best_model_path}")
            return
            
        final_save_path = os.path.join(save_path, model_names[model_type])
        
        # Copy the model file
        import shutil
        shutil.copy(best_model_path, final_save_path)
        print(f"Model saved to {final_save_path}")
        
    except Exception as e:
        print(f"Error saving model: {str(e)}")

# Save models with appropriate type
#save_trained_model(
#    sag_t1_model.model, 
#    '/kaggle/working/spine_detection/sagittal_t1_yolo/weights/best.pt',
#    model_type='sag_t1'
#)

save_trained_model(
    sag_t2_model.model, 
    '/kaggle/working/spine_detection/sagittal_t2_yolo/weights/best.pt',
    model_type='sag_t2'
)

#save_trained_model(
#    axial_t2_model.model, 
#    '/kaggle/working/spine_detection/axial_t2_yolo/weights/best.pt',
#    model_type='axial_t2'
#)

In [ ]:
!rm -r yolov8x.pt
!rm -r yolo11n.pt

In [ ]:
# Remove all the folders execpt the weights
import shutil
shutil.rmtree("spine_detection")
#shutil.rmtree("spine_dataset_sagt1")
shutil.rmtree("spine_dataset_segt2")
#shutil.rmtree("spine_dataset_axialt2")

In [ ]:
def plot_spine_predictions(image_path, model_path, 
                         conf_threshold=0.25, iou_threshold=0.45, img_size=384):
    """
    Plot YOLO predictions for spine levels on a DICOM image
    
    Args:
        image_path: Path to DICOM image
        model_path: Path to saved YOLO model
        conf_threshold: Confidence threshold for predictions
        iou_threshold: IOU threshold for NMS
        img_size: Image size for model input
    """
    # Load model
    model = YOLO(model_path)
    
    # Read DICOM
    ds = pydicom.dcmread(image_path)
    image = ds.pixel_array
    
    # Normalize and resize
    image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    image_resized = cv2.resize(image_normalized, (img_size, img_size))
    
    # Convert grayscale to RGB
    image_rgb = np.stack([image_resized] * 3, axis=-1)
    
    # Create figure
    plt.figure(figsize=(15, 7))
    
    # Plot original image
    plt.subplot(1, 2, 1)
    plt.imshow(image_resized, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    # Plot image with predictions
    plt.subplot(1, 2, 2)
    plt.imshow(image_resized, cmap='gray')
    plt.title('Predictions')
    
    # Get predictions
    results = model.predict(
        source=image_rgb,
        conf=conf_threshold,
        iou=iou_threshold
    )
    
    # Define colors for each level
    colors = ['red', 'green', 'blue', 'yellow', 'purple']
    level_names = ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']
    
    if results[0].boxes is not None:
        boxes = results[0].boxes.cpu().numpy()
        
        # Sort boxes by y-coordinate to display levels in order
        box_data = []
        for box in boxes:
            cls_id = int(box.cls[0])
            conf = box.conf[0]
            x1, y1, x2, y2 = box.xyxy[0]
            box_data.append((y1, cls_id, conf, x1, y1, x2, y2))
        
        box_data.sort()  # Sort by y1 coordinate
        
        # Plot each detection
        for i, (_, cls_id, conf, x1, y1, x2, y2) in enumerate(box_data):
            color = colors[cls_id]
            level_name = level_names[cls_id]
            
            # Draw bounding box
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                fill=False, color=color, linewidth=2
            ))
            
            # Add label
            plt.text(
                x2 + 5, (y1 + y2) / 2, 
                f'{level_name}: {conf:.2f}',
                color=color, fontsize=8, verticalalignment='center',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
            )
            
            # Print detection info
            print(f"Found {level_name} with confidence {conf:.2f}")
    else:
        print("No detections found")
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def process_multiple_images(image_paths, model_path, 
                          conf_threshold=0.25, iou_threshold=0.45):
    """
    Process multiple images and display their predictions, skipping images without predictions
    
    Args:
        image_paths: List of paths to DICOM images
        model_path: Path to saved YOLO model
        conf_threshold: Confidence threshold for predictions
        iou_threshold: IOU threshold for NMS
    """
    # First, filter images that have predictions
    valid_images = []
    model = YOLO(model_path)
    
    for img_path in image_paths:
        ds = pydicom.dcmread(img_path)
        image = ds.pixel_array
        image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        image_resized = cv2.resize(image_normalized, (384, 384))
        image_rgb = np.stack([image_resized] * 3, axis=-1)
        
        results = model.predict(
            source=image_rgb,
            conf=conf_threshold,
            iou=iou_threshold
        )
        
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            valid_images.append(img_path)
    
    if not valid_images:
        print("No images with valid predictions found")
        return
    
    # Process only images with predictions
    for img_path in valid_images:
        plot_spine_predictions(img_path, model_path, conf_threshold, iou_threshold)

In [ ]:
# Usage example for Sagittal T2:
image_paths = glob.glob('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/4646740/3666319702/*.dcm')
process_multiple_images(image_paths, '/kaggle/working/sagittal_t2_spine_detector.pt')